In [12]:
from datasets import load_dataset
import pandas as pd

from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.get_default_device()
img_to_tensor = transforms.ToTensor()
generator = torch.Generator(device)

batch_size = 100
n_classes = 10
embed_dim = 64

In [13]:
class ImageDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return img_to_tensor(self.data[idx]["img"]).to(device), torch.tensor(self.data[idx]["label"])
    
ds = load_dataset("uoft-cs/cifar10")

train_dl = DataLoader(ImageDataset(ds["train"]),batch_size=batch_size, shuffle=True, drop_last=True, generator=generator)
test_dl = DataLoader(ImageDataset(ds["test"]), batch_size=batch_size, shuffle=False, generator=generator)

In [14]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self,x):
        return self.layers(x) + self.shortcut(x)


class ResidualNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_head = nn.Sequential(
            nn.Conv2d(3, embed_dim//4, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim//4),
            nn.ReLU()
        )
        
        self.layer1 = ResidualBlock(embed_dim//4, embed_dim//2, stride=1)
        self.layer2 = ResidualBlock(embed_dim//2, embed_dim, stride=2)
        self.layer3 = ResidualBlock(embed_dim, embed_dim, stride=2)
        self.layer4 = ResidualBlock(embed_dim, embed_dim, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.out_head = nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        x = self.in_head(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.out_head(x)

In [15]:
epochs = 50 
model = ResidualNetwork()
opt = torch.optim.AdamW(model.parameters())
loss_fn = nn.CrossEntropyLoss()
history = []

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

Total parameters: 229,786


In [16]:

for epoch in range(epochs):
    total_loss = 0
    correct = 0

    for input_batch, label_batch in tqdm(train_dl):
        opt.zero_grad()

        pred_batch = model(input_batch)
        loss = loss_fn(pred_batch, label_batch)
        loss.backward()

        opt.step()

        with torch.no_grad():
            total_loss += loss.item()
            for i,label in enumerate(label_batch):
                if pred_batch[i,label.item()] == pred_batch[i].max():
                    correct+=1

    
    train_loss = total_loss/ len(train_dl)
    train_acc = correct / (len(train_dl) * batch_size)
    
    model.eval()
    test_loss, test_correct, m = 0, 0, 0

    with torch.no_grad():
        for input_batch, label_batch in test_dl:
            logits = model(input_batch)
            loss = loss_fn(logits, label_batch)

            test_loss += loss.item() * input_batch.size(0)
            preds = logits.argmax(dim=1)
            test_correct += (preds == label_batch).sum().item()
            m += input_batch.size(0)

    test_loss /= m
    test_acc = test_correct / m

    metrics = {
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'test_loss': test_loss,
        'test_acc': test_acc
    }
    print(metrics, "\n\n")
    # ---- Log metrics ----
    history.append(metrics)

    history_df = pd.DataFrame(history)
    history_df.to_csv("./history_resnet.csv", index=False)


    torch.save(model.state_dict(), "./resnet4_model.pth")

100%|██████████| 500/500 [00:29<00:00, 16.95it/s]


{'epoch': 1, 'train_loss': 1.2978455173969268, 'train_acc': 0.52948, 'test_loss': 1.15022882938385, 'test_acc': 0.579} 




100%|██████████| 500/500 [00:23<00:00, 21.71it/s]


{'epoch': 2, 'train_loss': 1.0609982424974442, 'train_acc': 0.61752, 'test_loss': 0.9472595107555389, 'test_acc': 0.662} 




100%|██████████| 500/500 [00:22<00:00, 21.89it/s]


...

{'epoch': 48, 'train_loss': 0.06985008442495018, 'train_acc': 0.97536, 'test_loss': 1.4041248989105224, 'test_acc': 0.7626} 




100%|██████████| 500/500 [00:22<00:00, 21.96it/s]


{'epoch': 49, 'train_loss': 0.06270915338210761, 'train_acc': 0.9776, 'test_loss': 1.433794322013855, 'test_acc': 0.7506} 




100%|██████████| 500/500 [00:23<00:00, 20.92it/s]


{'epoch': 50, 'train_loss': 0.07326164066977799, 'train_acc': 0.974, 'test_loss': 1.3633489269018173, 'test_acc': 0.7557} 




In [17]:
import plotly.express as px

history_df = pd.read_csv("./history_resnet.csv")

fig = px.line(history_df, x="epoch", y=["train_loss", "test_loss"])
fig.show()

fig = px.line(history_df, x="epoch", y=["train_acc", "test_acc"])
fig.show()